In [1]:
import pandas as pd
import re

In [2]:
def to_scientific_notation(value, decimal_places=4):
    """
    숫자를 지정된 소수점 자릿수의 지수 표기법으로 변환합니다.
    """
    format_string = f'{{:.{decimal_places}e}}'
    return format_string.format(value)

def convert_column_to_scientific(df, column_name, decimal_places=4):
    """
    DataFrame의 특정 열을 지수 표기법으로 변환합니다.
    """
    df[column_name] = df[column_name].apply(lambda x: to_scientific_notation(x, decimal_places))
    return df

In [3]:
# general information
file_path = './kinetic_DB/'
i_list = list(range(1,250))
#i_list = [101]

In [4]:
# bolsigdb.dat 생성
with open('bolsigdb.dat','w') as file:
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        f = convert_column_to_scientific(f, 'Energy(eV)')
        f = convert_column_to_scientific(f, 'Cross_section(m2)')

        type = f['Type'].loc[0]
        reaction = f['Reaction'].loc[0]
        if type == 'ELASTIC' or 'EFFECTIVE':
            eloss = str(f['ELoss'].loc[0])+' / mass ratio'
        else:
            eloss = str(f['ELoss'].loc[0])+' / threshold energy'

        line = '-----------------------------'

        file.write(type+'\n')
        file.write(reaction+'\n')
        file.write(eloss+'\n')
        file.write(line+'\n')
        for j in range(len(f)):
            file.write('  '+str(f['Energy(eV)'].loc[j])+'  '+str(f['Cross_section(m2)'].loc[j])+'\n')
        file.write(line+'\n')
        file.write('\n')
    
    file.close()

In [5]:
# kinep.inp 생성
with open('kinet.inp','w') as file:
    species = []
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        species += f['Formula'].loc[0].split(' ')
    species = list(set(species))
    species.remove('+')
    species.remove('->')
    species.remove('E')
    species = ['e']+species

    if len(species) > 31:
        species1 = species[:31]
        species2 = species[31:]
    else:
        species1 = species
        species2 = []

    species1 = ' '.join(species1)
    species2 = ' '.join(species2)    

    file.write('ELEMENTS'+'\n')
    file.write('e C H'+'\n')
    file.write('END'+'\n')
    file.write('\n')
    file.write('SPECIES'+'\n')
    file.write(species1+'\n')
    if species2 != []:
        file.write(species2+'\n')
    file.write('END'+'\n')
    file.write('\n')

    bolsig = []
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        bolsig += f['Species'].loc[0].split(' ')
    bolsig = list(set(bolsig))
    bolsig.remove('/')
    bolsig.remove('e')
    bolsig = ' '.join(bolsig)
    bolsig = bolsig
    
    file.write('BOLSIG'+'\n')
    file.write(bolsig+'\n')
    file.write('END'+'\n')
    file.write('\n')

    file.write('REACTIONS'+'\n')
    file.write('#'+'\n')
    file.write('# electron collision reaction'+'\n')
    file.write('#'+'\n')
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        if f['Type'].loc[0] != 'ELASTIC' and f['Type'].loc[0] != 'EFFECTIVE':
            formula = f['Formula'].loc[0]
            formula = formula.replace('E','e').replace('->','=>')
            formula = formula
            bolsig_comp = formula.replace('e','').replace('+','').replace('=>','->')
            bolsig_comp = bolsig_comp.split(' ')
            bolsig_comp = [item for item in bolsig_comp if item != '']
            bolsig_comp = ' '.join(bolsig_comp)
            if f['Type'].loc[0] == 'IONIZATION':
                bolsig_comp += '+'
            bolsig_comp = bolsig_comp
            file.write(formula)
            file.write('\t'*3)
            file.write('! Bolsig+ '+ bolsig_comp)
            file.write('\n')
    file.write('END')